This bugfix replaces the doi.long_name in the database with the cmr EntryTitle field

In [1]:
# this code allows us to run the internal packages

import os
import sys

parent_directory = os.path.split(os.getcwd())[0]

import_path = parent_directory + '/data_models'

if import_path not in sys.path:
    sys.path.insert(0, import_path)

In [8]:
import pickle
from api import Api

In [3]:
campaigns = [
    'above',
    'ACES',
    'AirMOSS',
    'ARCTAS',
    'CARVE',
    'GCPEx',
    'GOES-R PLT',
    'GRIP',
    'HS3',
    'OLYMPEX'
]

In [4]:
campaign_metadata = []
for campaign in campaigns:
    data = pickle.load(open(f'cmr_data-{campaign}', 'rb'))
    campaign_metadata.extend(data)
    print(campaign)
    print(len(data))

above
166
ACES
6
AirMOSS
21
ARCTAS
2
CARVE
28
GCPEx
44
GOES-R PLT
19
GRIP
13
HS3
15
OLYMPEX
36


In [35]:
api = Api('production')

In [36]:

successfully_replaced = []
for concept in campaign_metadata:
    cmr_doi = concept['metadata'].get('DOI',{}).get('DOI', 'FUCK'*50)
    cmr_short_name = concept['metadata']['ShortName']
    cmr_title = concept['metadata']['EntryTitle']
    
    if cmr_doi:
        doi_response = api.get(f'doi?short_name={cmr_doi}')
        doi_data = doi_response['data']
        if doi_data:
            doi_uuid = doi_data[0].get('uuid')
            doi_doi = doi_data[0].get('short_name')
            doi_long_name = doi_data[0].get('long_name')
            
#             assert doi_doi == cmr_doi
#             assert cmr_short_name == doi_long_name
            
            api.update(f'doi/{doi_uuid}', {'long_name': cmr_title})

            print(doi_uuid)
            print(f'replace {cmr_short_name}, {cmr_doi}\n   with {cmr_title}')
            print()            
            
            successfully_replaced.append(doi_uuid)        
    

14ec8bbe-cad5-46be-a5c9-45a28106aa5c
replace CH4_Aircraft_STILT_footprints_1300, 10.3334/ORNLDAAC/1300
   with CARVE-ARCSS: Methane Loss From Arctic- Fluxes From the Alaskan North Slope, 2012-2014

9f1bc12c-676a-4726-aa12-b3137d83a5e9
replace CARVE_Reports_1434, 10.3334/ORNLDAAC/1434
   with CARVE: Daily Flight Reports, 2012-2015

2858cae4-5af0-4b99-a29c-b9424fd47ae5
replace CARVE_Photos_1435, 10.3334/ORNLDAAC/1435
   with CARVE: In-flight Photos from the CARVE Aircraft, Alaska, 2013-2015

d178598a-6761-4197-937d-89b28f1a7158
replace CARVE_Videos_1433, 10.3334/ORNLDAAC/1433
   with CARVE: In-flight Videos from the CARVE Aircraft, Alaska, 2012-2015

b71cdc06-5871-4c50-a57b-63613aeb4aef
replace CARVE_L1_Infrared_1428, 10.3334/ORNLDAAC/1428
   with CARVE: L1 Airborne Forward Looking Infrared Radiance Counts, Alaska, 2013-2015

31545c67-4d00-4cfa-9f1d-b1c7a4aa9119
replace CARVE_L1_FlightPath_1425, 10.3334/ORNLDAAC/1425
   with CARVE: L1 Daily Flight Path Geolocation and Aircraft Position D

06698604-c930-4beb-a918-a6285606b126
replace goesrpltfegs, 10.5067/GOESRPLT/FEGS/DATA101
   with GOES-R PLT Fly's Eye GLM Simulator (FEGS) V1

c7821a80-ceb1-4c26-8e12-8119c60cd09b
replace goesrpltgcas, 10.5067/GOESRPLT/GCAS/DATA101
   with GOES-R PLT Geostationary Coastal and Air Pollution Event (GEO-CAPE) Airborne Simulator (GCAS) V1

6164ad7b-299a-40fa-999c-062a1ef2b9dc
replace goesrpltksclma, 10.5067/GOESRPLT/LMA/DATA301
   with GOES-R PLT Kennedy Space Center Lightning Mapping Array (LMA) V1

d2b37379-2d21-4411-8f11-92b7213753f6
replace goesrpltlip, 10.5067/GOESRPLT/LIP/DATA101
   with GOES-R PLT Lightning Instrument Package (LIP) V1

66d0f5fa-e5e2-41a7-8e12-b8b99f25221d
replace goesrpltnalma, 10.5067/GOESRPLT/LMA/DATA401
   with GOES-R PLT North Alabama Lightning Mapping Array (LMA) V1

a7eb26f6-0dc0-4d96-bee8-754d71f738a0
replace goesrpltoklma, 10.5067/GOESRPLT/LMA/DATA501
   with GOES-R PLT Oklahoma Lightning Mapping Array (LMA) V1

85511d62-04bd-4e5b-8fdb-691f8ec22016
replace g

In [32]:
len(set(successfully_replaced))

27